In [35]:
from google.colab import userdata
import os

OPENAI_API_KEY = userdata.get('OPEN_API_KEY')
TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')

# Set the API keys as environment variables
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

In [4]:
#!pip install langgraph
!pip install langchain-community --quiet
!pip install langchain --upgrade --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00


In [38]:
from tavily import TavilyClient

In [43]:
import os
from tavily import TavilyClient
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.chat_models import ChatOpenAI

# ---------------------------
# Initialize the LLM
# ---------------------------
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

# ---------------------------
# Step 1: Define the Tavily Search Tool using TavilyClient with Analysis
# ---------------------------
def tavilly_search(query: str) -> str:
    """
    Uses TavilyClient to fetch competitor data based on the query, then generates a detailed analysis report using an LLM.
    The API key is taken from the environment variable 'TAILY_API_KEY' or 'TAVILY_API_KEY'.
    """
    # Retrieve the API key from environment variables (using TAILY_API_KEY if set, otherwise TAVILY_API_KEY)
    api_key = os.getenv("TAILY_API_KEY", os.environ.get('TAVILY_API_KEY'))
    client = TavilyClient(api_key)
    try:
        # Fetch competitor data from Tavily
        response = client.search(query=query)

        # Build a prompt for the LLM to analyze the data
        analysis_prompt = (
            "You are an expert business analyst. Based on the competitor data below for clothing stores, "
            "provide a detailed analysis report that includes:\n"
            "- Identification of competitor clothing stores and their locations\n"
            "- Footfall trends and peak customer hours\n"
            "- Observations on customer behavior patterns\n"
            "- Actionable recommendations for improving market engagement\n\n"
            "Competitor Data:\n" + str(response)
        )

        # Generate the analysis report using the LLM
        analysis = llm.invoke(analysis_prompt)
        return analysis
    except Exception as e:
        return f"Error fetching data: {e}"

# Wrap the search function into a Langchain Tool.
tavilly_tool = Tool(
    name="TavillySearch",
    func=tavilly_search,
    description=(
        "Use this tool to fetch competitor data (e.g., nearby clothing store locations, footfall trends, and peak hours) "
        "from the Tavily Search API via TavilyClient and then generate a detailed analysis report using an LLM. "
        "It requires the TAILY_API_KEY for authentication."
    )
)

# ---------------------------
# Step 2: Initialize the Conversational Agent
# ---------------------------
agent = initialize_agent(
    tools=[tavilly_tool],
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True
)

# ---------------------------
# Step 3: Create a Chat Interface with Chat History
# ---------------------------
def chat_interface():
    chat_history = ""  # Initialize chat history to maintain context
    print("Welcome to the Competitive Clothing Store Analysis Chatbot!")
    print("Type 'exit' or 'quit' to end the session.")

    while True:
        user_input = input("\nYou: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Session ended.")
            break
        try:
            # Pass the user input and chat history to the agent.
            response = agent.run({"input": user_input, "chat_history": chat_history})
            print("\nBot:", response)
            # Update the chat history (simple concatenation; adjust format as needed)
            chat_history += f"User: {user_input}\nBot: {response}\n"
        except Exception as e:
            print(f"An error occurred: {e}")

# ---------------------------
# Step 4: (Optional) Visualize the Chain using Langgraph
# ---------------------------
try:
    from langgraph import LangGraph
    graph = LangGraph.from_agent(agent)
    graph.show()
except ImportError:
    print("Langgraph is not installed. Skipping graph visualization.")

# ---------------------------
# Step 5: Run the Chat Interface
# ---------------------------
if __name__ == "__main__":
    chat_interface()


Langgraph is not installed. Skipping graph visualization.
Welcome to the Competitive Clothing Store Analysis Chatbot!
Type 'exit' or 'quit' to end the session.

You: Pune Fashion Street


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: TavillySearch
Action Input: Pune Fashion Street
Observation: content="Analysis Report:\n\nIdentification of Competitor Clothing Stores and Their Locations:\nThe primary competitor identified from the data is 'Pune Fashion Street'. It is a renowned shopping spot in Pune, known for its variety of clothing options. The street market is located in one of the prime locations in the city, specifically at Mahatma Gandhi Road, Camp-411001.\n\nFootfall Trends and Peak Customer Hours:\nThe data does not provide specific information on footfall trends or peak customer hours. However, given that it is a popular shopping destination in a prime location, it can be inferred that the market likely experiences high footfall, especially